# Đồ án: Khoa học Dữ liệu và Ứng dụng

## Thông tin chung:
### Giảng viên: 
- Trần Trung Kiên
- Nguyễn Thái Vũ

### Thông tin nhóm:
1. STT: Nhóm 02
2. Thành viên:

| STT | MSSV | Họ tên | Github |
| --- | --- | --- | --- |
| 1 | 1712727 | Nguyễn Hoàng Sơn | [hsjune29th](https://github.com/hsjune29th) |
| 2 | 1712258 | Nguyễn Văn Hậu | [kenneth-nguyenn](https://github.com/kenneth-nguyenn) |
| 3 | 1712888 | Nguyễn Đình Tuyên | [Tuyen177](https://github.com/Tuyen177) |

3. Phân công công việc:
- [Google Sheet]()

## 1. Mô tả bài toán:

- **Tên cuộc thi:** [Google Brain - Ventilator Pressure Prediction](https://www.kaggle.com/competitions/ventilator-pressure-prediction/data?select=test.csv)
- **Mô tả:** Được tổ chức bởi Google Brain vào 04/11/2021 với hơn 2,605 nhóm tham dự.
- **Input:** Tập dữ liệu được thu thập từ máy thở cung cấp khí oxy cho phổi.
- **Output:** Dự đoán áp lực trên đường thở (pressure).
- **Ý nghĩa thực tiễn của ứng dụng:**
    + Thở máy là một biện pháp chuyên sâu của bác sĩ lâm sàng, dưới sự khẩn thiết của đại dịch COVID-19 trên diện rộng đã chỉ ra điểm hạn chế của việc sử sụng phương pháp thở máy này. Việc phát triển các phương pháp mới để điều khiển máy thở cơ học rất tốn kém, gặp khó khăn khi thử nghiệm lâm sàng.
    + Bài toán này cũng là tiền đề cho các thuật toán thích ứng với bệnh nhân, giúp giảm bớt gánh nặng cho bác sĩ lâm sàng trong tình hình mới. Do đó, các phương pháp điều trị bằng máy thở có thể được phổ biến rộng rãi hơn để giúp bệnh nhân thở được.

## 2. Giải quyết bài toán:

### 2.1. Ý tưởng:
- Nhóm quyết định tìm hiểu và cài đặt giải pháp **"[The inverse of a PID controller](https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/285283)"**, giải pháp đứng vị thứ hai trong leaderboard.
- Qúa trình hoàn thành cuộc thi, nhóm tác giả thực hiện ba giai đoạn chính:
    + Sử dụng 7 model được share từ @dlaststark ([notebook](https://www.kaggle.com/code/dlaststark/gb-vpp-pulp-fiction/notebook)). Trong notebook này, tác giả trên tập trung vào `Feature Engineering` và xây dựng `
Keras DNN Model` trên TPU.
    + Tiếp theo, tác gỉa tập hợp các dự đoán của 7 model trên bằng cách kết hợp diểm mạnh của giá trị median và mean [tại đây](https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/282735).
    + Cuối cùng, tác giả cố gắng xác định hàm nghịch đảo của PID controller (tìm nghịch đảo của bộ điều khiển P và sau đó chuyển sang bộ điều khiển PI) để dự đoán áp suất, được thảo luận [tại đây](https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/285283).

### 2.2. Lý do lựa chọn:
- Giải pháp đứng vị thứ hai, nằm trong trong TOP-10 leaderboard.
- Hầu hết, các giải pháp đứng trong TOP đầu của leaderboard đều sử dụng đến **PID controller** để giải quyết bài toán này. Mặt khác, trong quá trình đọc phần notebook cũng như giải thích của tác giả, nhóm em cảm thấy dễ hiểu và nắm được cơ bản quá trình xử lý trong notebook.

## 3. Somethings:

## 4. Nhìn lại quá trình làm đồ án:

### 4.1. Nguyễn Hoàng Sơn:

- Khó khăn:
    + 
    +
    +

- Học được:
    + 
    +
    +

### 4.2. Nguyễn Văn Hậu:

- Khó khăn:
    + 
    +
    +

- Học được:
    + 
    +
    +

### 4.3. Nguyễn Đình Tuyên:

- Khó khăn:
    + 
    +
    +

- Học được:
    + 
    +
    +

### 4.4. Nhóm:

- Khó khăn:
    + 
    +
    +

- Nếu có thêm thời gian, nhóm sẽ cố gắng thực hiện:
    + 
    +
    +

## 5. Tài liệu tham khảo:

Để hoàn thành đồ án này, nhóm mình đã tham khảo những tài liệu sau:
1. https://quantri.nhidong.org.vn/Data/bvnhidong/bvnhidong/Attachments/2019_3/07_theo_doi_benh_nhan_tho_may_-_2015-04_263201916.pdf
